In [25]:
import pandas as pd
import numpy as np
import statistics

## Loading metrics

In [44]:
samples = pd.read_csv("../1-GettingQuestions/sampleQuestions.csv")
samples["full_name"] = samples["framework"] + "/" +samples["path"]
samples = samples["full_name"].unique()

In [45]:
def loadMetrics(listSamples):
    metrics = dict()
    for sample in listSamples:
        metrics[sample] = pd.read_csv(f"../3-DataMerge/{sample}.csv", index_col=[0,1], na_filter=False)
    return metrics

In [46]:
normalized_metrics_questions = loadMetrics(samples)

## Mann Kendall Test

Null Hypothesis: There is no monotonic trend.

Alternative Hypothesis: There is a trend.

If p-value < 0.05 then null Hypothesis is rejected.

In [47]:
pip install pymannkendall

Note: you may need to restart the kernel to use updated packages.


In [48]:
import pymannkendall as mk

In [74]:
def extractMannKendallTest(metrics, listSamples):
    result = pd.DataFrame(index=listSamples)
    result.index.rename("full_name", inplace=True)
    for sample in listSamples:
        sampleMetric = metrics[sample]
        for metric in sampleMetric.columns.values:
            mkResult = mk.original_test(sampleMetric[metric])
            result.loc[sample, metric+" trend"] = mkResult.trend
            result.loc[sample, metric+" pvalue"] = mkResult.p
            result.loc[sample, "framework"] = sample.split("/")[0]
    return result

In [75]:
mannKendallResult = extractMannKendallTest(normalized_metrics_questions, normalized_metrics_questions.keys())

In [76]:
mannKendallResult.to_csv("mannKendallTest/mann_kendall_results.csv")

## Normality Test (Shapiro-Wilk)

Null Hyphotesis: The population has normal distribution

Alternative Hyphotesis: The population has not normal distribution

If p-value < 0.05 then null hyphotesis is rejected

## Correlation Test

Null Hypothesis: there is no correlation between data

Alternative Hypothesis: there is a correlation

If p-value < 0.05 then null Hypothesis is rejected.

If the metric is normalized we will apply the Pearson Test, otherwise we will apply Spearman Test

In [77]:
from scipy import stats

In [83]:
def extractCorrelationTest(metrics, listSamples):
    result = pd.DataFrame(index=listSamples)
    result.index.rename("full_name", inplace=True)
    for sample in listSamples:
        sampleMetric = metrics[sample]
        for metric in sampleMetric.columns.values:
            #test if the set has normal distribution
            shapiroPvalue = stats.shapiro(sampleMetric[metric]).pvalue
            if(shapiroPvalue < 0.05):
                correlationResult = stats.spearmanr(sampleMetric["questions"], sampleMetric[metric], nan_policy="omit")    
            else:
                correlationResult = stats.pearsonr(sampleMetric["questions"], sampleMetric[metric])
            result.loc[sample, metric+" correlation"] = correlationResult[0]
            result.loc[sample, metric+" pvalue"] = correlationResult[1]
            result.loc[sample, "framework"] = sample.split("/")[0]

    return result

In [84]:
correlationResult = extractCorrelationTest(normalized_metrics_questions, normalized_metrics_questions.keys())

In [85]:
correlationResult.to_csv("correlationTest/correlation_results.csv")

In [17]:
pip install PyGithub

     |████████████████████████████████| 291 kB 828 kB/s eta 0:00:01
     |████████████████████████████████| 961 kB 1.7 MB/s eta 0:00:01
  Created wheel for wrapt: filename=wrapt-1.12.1-cp38-cp38-linux_x86_64.whl size=81770 sha256=1e512685d67f5f17ebcbb91303386f80ea93880cb8d6dfd37fbfc36c39510152
  Stored in directory: /home/jovyan/.cache/pip/wheels/5f/fd/9e/b6cf5890494cb8ef0b5eaff72e5d55a70fb56316007d6dfe73
Successfully built wrapt
Note: you may need to restart the kernel to use updated packages.


In [18]:
from github import Github

In [19]:
g = Github("9e3a972c36e562286ad8ce5b189419f064234303")

In [68]:
def printAgeOfCodeSamplesWithIncreasingReadability(increasings, mannKendallResult):
    increasing = mannKendallResult[mannKendallResult["Readability trend"] == "increasing"]["Readability trend"].index.values
    for sample in increasing:
        year = g.get_repo(sample).created_at.strftime("%Y")
        print(sample, year)
        increasings.append(int(year))
    return increasings

In [69]:
increasings = []
increasings = printAgeOfCodeSamplesWithIncreasingReadability(increasings, androidMannKendallResult)
increasings = printAgeOfCodeSamplesWithIncreasingReadability(increasings, awsMannKendallResult)    
increasings = printAgeOfCodeSamplesWithIncreasingReadability(increasings, azureMannKendallResult)
increasings = printAgeOfCodeSamplesWithIncreasingReadability(increasings, springMannKendallResult)    

googlesamples/android-AutofillFramework 2017
googlesamples/google-services 2015
googlesamples/easypermissions 2015
googlesamples/android-testing 2014
googlesamples/android-credentials 2015
googlesamples/android-testdpc 2015
googlesamples/android-ActivitySceneTransitionBasic 2014
googlesamples/android-DirectBoot 2016
googlesamples/androidtv-Leanback 2014
googlesamples/android-fit 2014
googlesamples/android-BasicRenderScript 2014
googlesamples/android-TextLinkify 2014
googlesamples/android-SynchronizedNotifications 2014
googlesamples/android-SlidingTabsBasic 2014
googlesamples/android-RuntimePermissions 2015
googlesamples/android-RevealEffectBasic 2014
googlesamples/android-NfcProvisioning 2015
googlesamples/android-NetworkConnect 2014
googlesamples/android-MediaRouter 2014
googlesamples/android-MediaBrowserService 2014
googlesamples/android-DirectorySelection 2014
googlesamples/android-BatchStepSensor 2014
googlesamples/android-BasicSyncAdapter 2014
googlesamples/android-play-safetynet 

In [92]:
statistics.mode(increasings)

2014

In [85]:
def printAgeOfCodeSamplesWithoutIncreasingReadability(noincreasings, mannKendallResult):
    noincreasing = mannKendallResult[mannKendallResult["Readability trend"] != "increasing"]["Readability trend"].index.values
    for sample in noincreasing:
        year = g.get_repo(sample).created_at.strftime("%Y")
        print(sample, year)
        noincreasings.append(int(year))
    return noincreasings

In [86]:
noincreasings = []
noincreasings = printAgeOfCodeSamplesWithoutIncreasingReadability(noincreasings, androidMannKendallResult)
noincreasings = printAgeOfCodeSamplesWithoutIncreasingReadability(noincreasings, awsMannKendallResult)    
noincreasings = printAgeOfCodeSamplesWithoutIncreasingReadability(noincreasings, azureMannKendallResult)
noincreasings = printAgeOfCodeSamplesWithoutIncreasingReadability(noincreasings, springMannKendallResult)    

googlesamples/android-play-location 2014
googlesamples/android-Camera2Basic 2014
googlesamples/android-EmojiCompat 2017
googlesamples/android-instant-apps 2017
googlesamples/android-nearby 2015
googlesamples/android-AppUsageStatistics 2014
googlesamples/android-custom-lint-rules 2015
googlesamples/android-play-billing 2015
googlesamples/android-vision 2015
googlesamples/android-ads 2015
googlesamples/android-BluetoothLeGatt 2014
googlesamples/android-Camera2Raw 2015
googlesamples/android-PictureInPicture 2017
googlesamples/android-Camera2Video 2014
googlesamples/android-XYZTouristAttractions 2015
googlesamples/android-WearVerifyRemoteApp 2016
googlesamples/android-WatchFace 2014
googlesamples/android-MediaEffects 2014
googlesamples/leanback-showcase 2016
googlesamples/androidtv-sample-inputs 2014
googlesamples/android-play-publisher-api 2014
googlesamples/android-BluetoothChat 2014
googlesamples/android-MediaRecorder 2014
googlesamples/android-DisplayingBitmaps 2014
googlesamples/andro

In [93]:
statistics.mode(noincreasings)

2014